In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/Text-Summarizer'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str
    output_file_name: str

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH) -> None:
        """
        Initializes the ConfigurationManager object.

        Args:
            config_filepath (str, optional): File path for the main configuration file.
            params_filepath (str, optional): File path for the parameters file.
        """

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_transformation_config(self) -> DataTransformationConfig:
        """
        Retrieves the data transformation configuration.

        Returns:
            DataTransformationConfig: Object containing data transformation configuration settings.
        """
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name,
            output_file_name = config.output_file_name,
        )

        return data_transformation_config

In [5]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

/home/izam/anaconda3/envs/text-summarizer/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-02-02 02:04:35,123: INFO: config: PyTorch version 2.2.0 available.]


In [10]:
class DataTransformation:
    """
    A class for performing data transformation tasks.

    Attributes:
        config (DataTransformationConfig): Configuration object for data transformation.
        tokenizer (AutoTokenizer): Tokenizer for processing text data.

    Methods:
        __init__: Initializes the DataTransformation object.
        _convert_examples_to_features: Converts examples to features for the given model.
        convert: Converts the input dataset to features and saves the transformed dataset.

    Usage:
        data_transformation = DataTransformation(config)
        data_transformation.convert()
    """
    
    def __init__(self, config:DataTransformationConfig) -> None:
        """
        Initializes the DataTransformation object.

        Args:
            config (DataTransformationConfig): Configuration object for data transformation.
        """
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    def _convert_examples_to_features(self, example_batch):
        """
        Converts examples to features for the given model.

        Args:
            example_batch (dict): Batch of examples containing "dialogue" and "summary".

        Returns:
            dict: Dictionary containing input_ids, attention_mask, and labels.
        """
        input_encodings =  self.tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch["summary"], max_length=128, truncation=True)
        
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }


    def convert(self):
        """
        Converts the input dataset to features and saves the transformed dataset.

        Returns:
            None
        """
        dataset_local = load_from_disk(self.config.data_path)
        dataset_local_pt = dataset_local.map(self._convert_examples_to_features, batched=True)
        dataset_local_pt.save_to_disk(os.path.join(self.config.root_dir, self.config.output_file_name))


In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e: 
    raise e

[2024-02-02 02:05:16,193: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-02 02:05:16,194: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-02 02:05:16,194: INFO: common: created directory at: artifacts]
[2024-02-02 02:05:16,195: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/home/izam/anaconda3/envs/text-summarizer/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 229694.09 examples/s]
